In [1]:
from streetview import search_panoramas
import folium
import numpy as np
from geopy.distance import geodesic

In [2]:
def find_nearby(lat, lon, lats_ls, lons_ls, panoid_ls):
  panos = search_panoramas(lat=lat, lon=lon)
  len(panos)


  # extract lat/lon from meta_ls
  for pano in panos:
      lats_ls.append(pano.lat)
      lons_ls.append(pano.lon)
      panoid_ls.append(pano.pano_id)

  return lats_ls, lons_ls, panoid_ls


  # Display map

In [3]:
center_lat, center_lon = 34.272, -118.484869
side_length_meters = 50
grids = 10  

radius_range_meters = 3

lats_ls, lons_ls, panoid_ls = [], [], []

In [4]:
# Calculate points 50m north and east of center using geodesic
north = geodesic(meters=side_length_meters).destination((center_lat, center_lon), 0)
east = geodesic(meters=side_length_meters).destination((center_lat, center_lon), 90)

# Extract lat/lon from the points
max_lat = north.latitude
max_lon = east.longitude

# Calculate grid cell sizes
grid_size_lat = max_lat - center_lat
grid_size_lon = max_lon - center_lon

grid_size_lat, grid_size_lon

(0.000450745617506243, 0.0005429513391703722)

In [5]:
# grid_size = 0.001  # About 100m in each direction

# num_points = grids/2
for i in range(int(-grids//2), int(grids//2) + 1):
    for j in range(int(-grids//2), int(grids//2) + 1):
        lat = center_lat + (i * grid_size_lat)
        lon = center_lon + (j * grid_size_lon)
        lats_ls, lons_ls, panoid_ls = find_nearby(lat, lon, lats_ls, lons_ls, panoid_ls)

In [6]:
# Create grid cells based on radius_range

# Using pyproj for more precise geodetic calculations

# Calculate points 3m north and east of center
north2 = geodesic(meters=radius_range_meters).destination((center_lat, center_lon), 0)
east2 = geodesic(meters=radius_range_meters).destination((center_lat, center_lon), 90)

# Extract lat/lon from the points
max_lat2 = north2.latitude
max_lon2 = east2.longitude

# Calculate the lat/lon ranges
radius_range_lat = max_lat2 - center_lat 
radius_range_lon = max_lon2 - center_lon

radius_range_lat, radius_range_lon

(2.7044737997528046e-05, 3.2577080347095944e-05)

In [7]:


lats_sorted = sorted(lats_ls)
lons_sorted = sorted(lons_ls)
lat_min, lat_max = lats_sorted[0], lats_sorted[-1]
lon_min, lon_max = lons_sorted[0], lons_sorted[-1]

print(lat_min)
print(lat_max)

print(lon_min)
print(lon_max)

34.26924215167309
34.27469364666767
-118.4883451283203
-118.4816070475573


In [8]:
# Create map centered on first point
m = folium.Map(location=[lats_ls[0], lons_ls[0]], zoom_start=16)

# Add markers for each point
for lat, lon in zip(lats_ls, lons_ls):
    folium.Marker([lat, lon]).add_to(m)

# Display the map
m

In [9]:

# Calculate cell sizes
lat_cells = (lat_max - lat_min) // radius_range_lat + 1
lon_cells = (lon_max - lon_min) // radius_range_lon + 1
# lat_cells = side_length_meters // radius_range_meters + 1
# lon_cells = side_length_meters // radius_range_meters + 1

# Initialize grid to store points
grid_points = [[[] for _ in range(int(lat_cells))] for _ in range(int(lon_cells))]
print(f"Grid points shape: {len(grid_points)} x {len(grid_points[0])} cells")

# Categorize points into grid cells
for lat, lon, panoid in zip(lats_ls, lons_ls, panoid_ls):
    # Calculate grid indices based on relative position
    lat_idx = int((lat - lat_min) // radius_range_lat) 
    lon_idx = int((lon - lon_min) // radius_range_lon)
    
    # # Handle edge case where point is exactly at max boundary
    # if lat_idx == radius_range:
    #     lat_idx = lat_cells - 1
    # if lon_idx == radius_range:
    #     lon_idx = radius_range - 1
        
    # Only add if indices are valid
    # if 0 <= lat_idx < radius_range and 0 <= lon_idx < radius_range:
    #     grid_points[lat_idx][lon_idx].append((lat, lon))
    # print(lat_idx, " ", lon_idx)
    try:
        grid_points[lon_idx][lat_idx].append((lon, lat, panoid))
    except:
        print(lon_idx, "", lat_idx)


# # Create map centered on first point
# m = folium.Map(location=[lats_ls[0], lons_ls[0]], zoom_start=16)

# # Add markers for each point
# for lat, lon in zip(lats_ls, lons_ls):
#     folium.Marker([lat, lon]).add_to(m)


Grid points shape: 207 x 202 cells


In [19]:
# Create lookup dictionary for panoid -> [lon, lat]
panoid_to_coords_dict = {}
for lon, lat, panoid in zip(lons_ls, lats_ls, panoid_ls):
    panoid_to_coords_dict[panoid] = [lon, lat]


In [16]:
# Initialize list to store points and their neighbors within range
points_with_neighbors = []
p2c_dict = {} #key: pano id, value: group leader's coordinates [lat, lon]
grouping_dict = {} #gets the grouping leader's coordinates, key: "lat" +"," + "lon", value: [all pano ids]

# Helper function to calculate distance between two points
def nearby(lat1, lon1, lat2, lon2, radius_range_meters):
    R = 6371000  # Earth's radius in meters
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c <= radius_range_meters

# Iterate through each cell in the grid
for i in range(len(grid_points)):
    for j in range(len(grid_points[0])):
        # Get points in current cell
        current_points = grid_points[i][j]
        
        # Skip if no points in current cell
        if not current_points:
            continue
            
        # Check adjacent cells (including diagonals)
        for di in [-1, 0, 1]:
            for dj in [-1, 0, 1]:
                ni, nj = i + di, j + dj
                
                # Skip if indices are out of bounds
                if not (0 <= ni < len(grid_points) and 0 <= nj < len(grid_points[0])):
                    continue
                    
                # Get points in neighbor cell
                neighbor_points = grid_points[ni][nj]
                
                # For each point in current cell
                for point1 in current_points:
                    # Compare with each point in neighbor cell
                    for point2 in neighbor_points:
                        # # Skip if same point
                        # if point1 == point2:
                        #     continue
                        p1lonlat_str = str(point1[0]) + "," + str(point1[1])
                        p2lonlat_str = str(point2[0]) + "," + str(point2[1])

                        if point1[2] in p2c_dict and point2[2] in p2c_dict:
                            pass
                        elif point1[2] in p2c_dict:
                            grouplon, grouplat = p2c_dict[point1[2]]
                            if nearby(grouplon, grouplat, point2[0], point2[1], radius_range_meters):
                                glonlat_str = str(grouplon) + "," + str(grouplat)
                                grouping_dict[glonlat_str].append(point2[2])
                                p2c_dict[point2[2]] = [grouplon, grouplat]
                        elif point2[2] in p2c_dict:
                            grouplon, grouplat = p2c_dict[point2[2]]
                            if nearby(point1[0], point1[1], grouplon, grouplat, radius_range_meters):
                                glonlat_str = str(grouplon) + "," + str(grouplat)
                                grouping_dict[glonlat_str].append(point1[2])
                                p2c_dict[point1[2]] = [grouplon, grouplat]
                        else:
                            if nearby(point1[0], point1[1],point2[0], point2[1], radius_range_meters):
                                grouping_dict[p1lonlat_str] = [point1[2], point2[2]]
                                grouplon, grouplat = point1[0], point1[1]
                                p2c_dict[point1[1]] = [grouplon, grouplat]
                                p2c_dict[point2[2]] = [grouplon, grouplat]


                        

                        # # Calculate actual distance
                        # distance = haversine_distance(point1[0], point1[1], point2[0], point2[1])
                        
                        # # If within range, add to results
                        # if distance <= radius_range_meters:
                        #     points_with_neighbors.append((point1, point2, distance))

# print(f"Found {len(points_with_neighbors)} point pairs within {radius_range_meters} meters")


Found 0 point pairs within 3 meters


In [21]:
print(f"Number of groups in grouping_dict: {len(grouping_dict)}")

# Create a base map centered on the first point
first_group = list(grouping_dict.values())[0]
first_pano = first_group[0]
first_coords = panoid_to_coords_dict[first_pano]
m = folium.Map(location=[first_coords[1], first_coords[0]], zoom_start=15)

# Generate a list of distinct colors for different groups
import random
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 
          'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 
          'darkpurple', 'pink', 'lightblue', 'lightgreen']

# Plot each group with a different color
for i, (center_str, pano_ids) in enumerate(grouping_dict.items()):
    # Get color for this group (cycle through colors if more groups than colors)
    color = colors[i % len(colors)]
    
    # Create a feature group for this cluster
    group = folium.FeatureGroup(name=f"Group {i}")
    
    # Plot each point in the group
    for pano_id in pano_ids:
        coords = panoid_to_coords_dict[pano_id]
        folium.CircleMarker(
            location=[coords[1], coords[0]],
            radius=5,
            color=color,
            fill=True,
            popup=f"Pano ID: {pano_id}",
        ).add_to(group)
    
    # Add the feature group to the map
    group.add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

m


Number of groups in grouping_dict: 646
